In [1]:
import os
import pysam
from pathlib import Path
import pandas as pd

In [2]:
# 过滤长度小于300bp的orf
for path in list(Path("05.predict/prodigal/").glob("*.ffn")):
    out_file = open(str(path).replace("ffn","300bp.fa"),"w")
    with pysam.FastxFile(str(path)) as fa:
        for seq in fa:
            if len(seq.sequence) >= 300:
                out_file.write("seq.name" + " " + seq.comment + "\n" + seq.sequence + "\n")
out_file.close()

In [70]:
# 合并GTDB的序列，并修改名字好
import os
import pysam
from pathlib import Path

out_file = open("/data3/wangjiaxuan/refer/GTDBTk_db_65703sp.fa","w")
with open("/data3/wangjiaxuan/refer/metapi_db/GTDBTk_db/release207_v2/taxonomy/gtdb_taxonomy_test.tsv","r") as tax_list:
    for line in tax_list:
        asid = line.split("\t")[0].replace(".1","")
        asid = asid.split("_")
        asid_number = [asid[2][i*3:i*3+3] for i in range(3)]
        path1 = os.path.join("/data3/wangjiaxuan/refer/metapi_db/GTDBTk_db/release207_v2/fastani/database",
                           asid[1],
                           asid_number[0],
                           asid_number[1],
                           asid_number[2])
        fa_file = Path(path1).glob("*.gz")
        with pysam.FastxFile(str(list(fa_file)[0])) as fa:
            for seq in fa:
                #print("_".join([asid[0],asid[1]]) + "_" + "".join(asid_number)+ ".1" + " " + seq.name + " " + seq.comment + "\n")
                out_file.write(">"+"_".join([asid[0],asid[1]]) + "_" + "".join(asid_number)+ ".1" + " " + seq.name + " " + seq.comment + "\n" + seq.sequence + "\n")
out_file.close()

In [28]:
# 合并samtools的统计结果

# function

import os
import os
import re
import pandas as pd
from decimal import *
import concurrent.futures
from pathlib import Path

def flagstats_summary(flagstats, method = 2, **kwargs):
    """
    get alignment rate from sorted bam file
    samtools flagstat --threads 8 sample.sort.bam
    """
    mapping_info = []
    getcontext().prec = 8

    # with open(flagstat_list, 'r') as list_handle:
    if method == 1:
        list_handle = open(flagstats, "r")
    if method == 2:
        list_handle = flagstats

    for flagstat_file in list_handle:
        if os.path.exists(flagstat_file.strip()):
            info = {}
            info["sample_id"] = os.path.basename(flagstat_file.strip()).split(".")[0]
            stat_list = open(flagstat_file.strip(), "r").readlines()

            info["total_num"] = stat_list[0].split(" ")[0]

            if len(stat_list) == 13:
                info["read_1_num"] = stat_list[6].split(" ")[0]
                info["read_2_num"] = stat_list[7].split(" ")[0]

                mapped = re.split(r"\(|\s+", stat_list[4])
                info["mapped_num"] = mapped[0]
                info["mapped_rate"] = Decimal(mapped[5].rstrip("%")) / Decimal(100)
            
                #primary_mapped = re.split(r"\(|\s+", stat_list[5])
                #info["primary_mapped_num"] = primary_mapped[0]
                #info["primary_mapped_rate"] = Decimal(primary_mapped[6].rstrip("%")) / Decimal(100)

            elif len(stat_list) == 16:
                info["read_1_num"] = stat_list[9].split(" ")[0]
                info["read_2_num"] = stat_list[10].split(" ")[0]

                mapped = re.split(r"\(|\s+", stat_list[6])
                info["mapped_num"] = mapped[0]
                info["mapped_rate"] = Decimal(mapped[5].rstrip("%")) / Decimal(100)
            
                primary_mapped = re.split(r"\(|\s+", stat_list[7])
                info["primary_mapped_num"] = primary_mapped[0]
                info["primary_mapped_rate"] = Decimal(primary_mapped[6].rstrip("%")) / Decimal(100)
 
            paired = re.split(r"\(|\s+", stat_list[-5])
            info["paired_num"] = paired[0]
            paired_rate = paired[6].rstrip("%")
            if paired_rate != "N/A":
                info["paired_rate"] = Decimal(paired_rate) / Decimal(100)
                info["mapping_type"] = "paired-end"
            else:
                info["paired_rate"] = paired_rate
                info["mapping_type"] = "single-end"

            singletons = re.split(r"\(|\s+", stat_list[-3])
            info["singletons_num"] = singletons[0]
            singletons_rate = singletons[5].rstrip("%")
            if singletons_rate != "N/A":
                info["singletons_rate"] = Decimal(singletons_rate) / Decimal(100)
            else:
                info["singletons_rate"] = singletons_rate

            info["mate_mapped_num"] = re.split(r"\(|\s+", stat_list[-2])[0]
            info["mate_mapped_num_mapQge5"] = re.split(r"\(|\s+", stat_list[-1])[0]
            mapping_info.append(info)

    mapping_info_df = pd.DataFrame(mapping_info)
    if "output" in kwargs:
        mapping_info_df.to_csv(kwargs["output"], sep="\t", index=False)
    return mapping_info_df

in_file_list = [str(x) for x in list(Path("../02.rmhost").glob("*.align2host.flagstat"))]

sum_flagstat= flagstats_summary(in_file_list)

sum_flagstat.to_csv("../02.rmhost/mapping_host_stat.csv")

In [46]:
# merge the bam flagstat
trim_fastp_stat = open("01.fastq_qc/fastq_trim_stats.txt","w")
with open("01.fastq_qc/fastp_multiqc_report_data/multiqc_general_stats.txt","r") as f:
    i = 1
    for line in f:
        if i == 1:
            l = line.replace("fastp_mqc-generalstats-fastp-","").title()
        else:
            l = line
        trim_fastp_stat.write(l)
        i += 1
trim_fastp_stat.close()

In [47]:
# merge fastp QC result

import os
import concurrent.futures
import pandas as pd


def parse(stats_file,sep = "\t"):
    if os.path.exists(stats_file):
        try:
            df = pd.read_csv(stats_file, sep = sep)
        except pd.errors.EmptyDataError:
            print("%s is empty, please check" % stats_file)
            return None

        if not df.empty:
            return df
        else:
            return None
    else:
        print("%s is not exists" % stats_file)
        return None


def merge(input_list, func, workers = 4, **kwargs):
    df_list = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
        for df in executor.map(func, input_list):
            if df is not None:
                df_list.append(df)

    df_ = pd.concat(df_list)

    if "output" in kwargs:
        df_.to_csv(kwargs["output"], sep="\t", index=False)
    return df_


In [58]:
# 01.fastq_qc
fastp_file_list = [x for x in list(Path("01.fastq_qc/").glob("*_trimming_stats.tsv"))]
fastp_stat = merge(fastp_file_list,parse,8)
fastp_stat.to_csv("01.fastq_qc//trimming_fastp_stat.csv")
# 02.rmhost
fastp_file_list = [x for x in list(Path("02.rmhost/").glob("*_rmhost_stats.tsv.raw"))]
fastp_stat = merge(fastp_file_list,parse,8)
fastp_stat.to_csv("02.rmhost/rmhost_fastp_stat.csv")

In [4]:
os.chdir("../")

In [80]:
# 加载
import re
from collections import defaultdict
'''
这是为了将输入的总表，按照分类级别输出表格
'''
# 词典
taxonomy_level = [
    "domain",
    "superkingdom",
    "phylum",
    "class",
    "order",
    "family",
    "genus",
    "species",
    "strain",
]

taxonomy_dict = {
    "strain": r"t__(.*?)[\|\t]",
    "species": r"s__(.*?)[\|\t]",
    "genus": r"g__(.*?)[\|\t]",
    "family": r"f__(.*?)[\|\t]",
    "order": r"o__(.*?)[\|\t]",
    "class": r"c__(.*?)[\|\t]",
    "phylum": r"p__(.*?)[\|\t]",
    "superkingdom": r"k__(.*?)[\|\t]",
    "domain" : r"d__(.*?)[\|\t]",
}

level_dict = {
    "strain": "t",
    "species": "s",
    "genus": "g",
    "family": "f",
    "order": "o",
    "class": "c",
    "phylum": "p",
    "superkingdom": "k",
    "domain" : "d"
}

# 输入
taxonomy_table = open("test.tsv","r")
# 输出
level_output_file = defaultdict(list)
level_output_header = dict()

for line in taxonomy_table:
    # 拆分分类和表达量
    taxonomy_info = line.split("\t")[0]
    exp_info = line.split("\t")[1:]
    taxonomy_name = []
    # header的输出
    if "clade_name" in line and "#" not in line:
        level_output_header['header'] = "\t".join(taxonomy_level)+"\t"+"\t".join(exp_info)
    elif "#" not in line:
        # 名字输出
        final_taxonomy_var = "d__unclassified"
        for lev in taxonomy_level:
            re_result = re.findall(taxonomy_dict[lev],line)
            if re_result:
                taxonomy_name.append(level_dict[lev]+"__"+re_result[0])
                final_taxonomy_var = lev
            else:
                taxonomy_name.append(level_dict[lev]+"__"+"unclassified")
        level_output_file['all'].append("\t".join(taxonomy_name)+"\t"+"\t".join(exp_info))
        level_output_file[final_taxonomy_var].append("\t".join(taxonomy_name)+"\t"+"\t".join(exp_info))

for lev in level_output_file:
    with open(f'03.taxonomy/all_sample_taxonomy_profile_{lev}.xls',"w") as f:
        f.write(level_output_header['header'])
        for line in level_output_file[lev]:
            f.write(line)
    f.close()